In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import pandas as pd
import datasets

/home/kldooste/miniconda3/envs/cebab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from data_utils import preprocess_hf_dataset 
from abstract_model import Model
from abstract_explainer import Explainer
from pipeline import cebab_pipeline, run_pipelines

In [5]:
from explainers.lime import LIME
from explainers.conexp import CONEXP
from explainers.causalm import CAUSALM

ModuleNotFoundError: No module named 'eval_pipeline'

In [12]:
from models.simple_lr import simple_lr
from models.roberta import RoBERTaForCEBaB
from models.bert import BERTForCEBaB
from models.lstm import LSTMForCEBaB
from models.gpt2 import GPT2ForCEBaB


### Binary/Ternary pipeline with pretrained models

BERT models

In [107]:
model_path = 'CEBaB/bert-base-uncased.CEBaB.sa.2-class.exclusive.seed_99'
# model_path = 'CEBaB/bert-base-uncased.CEBaB.sa.3-class.exclusive.seed_99'
# model_path = 'CEBaB/bert-base-uncased.CEBaB.sa.5-class.exclusive.seed_99'
dataset_type = '2-way'
# dataset_type = '3-way'
# dataset_type = '5-way'

device = 'cuda:0'
batch_size = 32

# load data from HF
cebab = datasets.load_dataset('CEBaB/CEBaB', use_auth_token='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')
cebab['train'] = cebab['train_exclusive']
train, dev, test = preprocess_hf_dataset(cebab, one_example_per_world=False, verbose=1, dataset_type=dataset_type)

# define models
tf_model = BERTForCEBaB(model_path, device=device, batch_size=batch_size)
lr_model = simple_lr()

# fit models
tf_model.fit(train)
lr_model.fit(train)

models = [
    tf_model,
    tf_model,
    lr_model,
    lr_model,
]

explanators = [
    CONEXP(),
    LIME(),
    CONEXP(),
    LIME(),
]

final_ATE,final_CEBaB_metrics, final_CEBaB_per_aspect_direction ,final_estimates_per_aspect_direction = run_pipelines(models, explanators, train, dev, dataset_type = dataset_type)

Using custom data configuration CEBaB--CEBaB-65fc31a444dff443
Reusing dataset parquet (/home/kldooste/.cache/huggingface/datasets/parquet/CEBaB--CEBaB-65fc31a444dff443/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

/project/OpenTable/eval_pipeline/data_utils.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['review_majority'] = dataset['review_majority'].apply(lambda score: encoding[score])
/project/OpenTable/eval_pipeline/data_utils.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[aspect_columns] = train[aspect_columns].fillna('')


Dropping no majority reviews: 16.6382% of train dataset.
Dropped 391 examples with a neutral label.
Dropped 452 examples with a neutral label.
Dropped 461 examples with a neutral label.
fitting 2274 LR models...


100%|██████████| 2274/2274 [00:29<00:00, 78.21it/s]


fitting 2274 LR models...


100%|██████████| 2274/2274 [00:29<00:00, 78.04it/s]


In [108]:
final_ATE

iTE
intervention_type intervention_aspect_base intervention_aspect_counterfactual count                   
ambiance          Negative                 Positive                           70     [-0.1857, 0.1857]
                                           unknown                            87       [-0.069, 0.069]
                  Positive                 Negative                           70     [0.1857, -0.1857]
                                           unknown                            107    [0.0374, -0.0374]
                  unknown                  Negative                           87       [0.069, -0.069]
                                           Positive                           107    [-0.0374, 0.0374]
food              Negative                 Positive                           112    [-0.8036, 0.8036]
                                           unknown                            107    [-0.3178, 0.3178]
                  Positive                 Negative                           112    [0.8036, -0.8036]
                                           unknown                            125      [0.112, -0.112]
                  unknown                  Negative                           107    [0.3178, -0.3178]
                                           Positive                           125      [-0.112, 0.112]
noise             Negative                 Positive                           69     [-0.0435, 0.0435]
                                           unknown                            83       [-0.012, 0.012]
                  Positive                 Negative                           69     [0.0435, -0.0435]
                                           unknown                            82     [0.0122, -0.0122]
                  unknown                  Negative                           83       [0.012, -0.012]
                                           Positive                           82     [-0.0122, 0.0122]
service           Negative                 Positive                           84     [-0.2976, 0.2976]
                                           unknown                            88     [-0.1705, 0.1705]
                  Positive                 Negative                           84     [0.2976, -0.2976]
                                           unknown                            123    [0.0407, -0.0407]
                  unknown                  Negative                           88     [0.1705, -0.1705]
                                           Positive                           123    [-0.0407, 0.0407]

In [109]:
final_CEBaB_metrics

CEBaB score
bert-base-uncased.CEBaB.sa.2-class.exclusive.se... CONEXP mean       0.4936
                                                   LIME2  mean       0.9768
LR                                                 CONEXP mean       0.1864
                                                   LIME2  mean       0.8816

In [110]:
final_CEBaB_per_aspect_direction

bert-base-uncased.CEBaB.sa.2-class.exclusive.seed_99  \
                                                                                                                                  CONEXP   
                                                                                                                             CEBaB score   
intervention_type intervention_aspect_base intervention_aspect_counterfactual count                                                        
ambiance          Negative                 Positive                           70                                                0.6567     
                                           unknown                            87                                                0.4045     
                  Positive                 Negative                           70                                                0.6567     
                                           unknown                            107                                               0.3348     
                  unknown                  Negative                           87                                                0.4045     
                                           Positive                           107                                               0.3348     
food              Negative                 Positive                           112                                               0.4575     
                                           unknown                            107                                               0.6878     
                  Positive                 Negative                           112                                               0.4575     
                                           unknown                            125                                               0.6029     
                  unknown                  Negative                           107                                               0.6878     
                                           Positive                           125                                               0.6029     
noise             Negative                 Positive                           69                                                0.5253     
                                           unknown                            83                                                0.2688     
                  Positive                 Negative                           69                                                0.5253     
                                           unknown                            82                                                0.2654     
                  unknown                  Negative                           83                                                0.2688     
                                           Positive                           82                                                0.2654     
service           Negative                 Positive                           84                                                0.7780     
                                           unknown                            88                                                0.6002     
                  Positive                 Negative                           84                                                0.7780     
                                           unknown                            123                                               0.3703     
                  unknown                  Negative                           88                                                0.6002     
                                           Positive                           123                                               0.3703     

                                                                                                 \
                                                                

In [111]:
final_estimates_per_aspect_direction

bert-base-uncased.CEBaB.sa.2-class.exclusive.seed_99  \
                                                                                                                                           
                                                                                                                                   iCaCE   
intervention_type intervention_aspect_base intervention_aspect_counterfactual count                                                        
ambiance          Negative                 Positive                           70                                     [-0.2141, 0.2141]     
                                           unknown                            87                                     [-0.0805, 0.0805]     
                  Positive                 Negative                           70                                     [0.2141, -0.2141]     
                                           unknown                            107                                    [0.0466, -0.0466]     
                  unknown                  Negative                           87                                     [0.0805, -0.0805]     
                                           Positive                           107                                    [-0.0466, 0.0466]     
food              Negative                 Positive                           112                                    [-0.7399, 0.7399]     
                                           unknown                            107                                    [-0.3269, 0.3269]     
                  Positive                 Negative                           112                                    [0.7399, -0.7399]     
                                           unknown                            125                                    [0.0882, -0.0882]     
                  unknown                  Negative                           107                                    [0.3269, -0.3269]     
                                           Positive                           125                                    [-0.0882, 0.0882]     
noise             Negative                 Positive                           69                                     [-0.0231, 0.0231]     
                                           unknown                            83                                           [-0.0, 0.0]     
                  Positive                 Negative                           69                                     [0.0231, -0.0231]     
                                           unknown                            82                                     [0.0122, -0.0122]     
                  unknown                  Negative                           83                                           [0.0, -0.0]     
                                           Positive                           82                                     [-0.0122, 0.0122]     
service           Negative                 Positive                           84                                     [-0.2856, 0.2856]     
                                           unknown                            88                                     [-0.2044, 0.2044]     
                  Positive                 Negative                           84                                     [0.2856, -0.2856]     
                                           unknown                            123                                    [0.0325, -0.0325]     
                  unknown                  Negative                           88                                     [0.2044, -0.2044]     
                                           Positive                           123                                    [-0.0325, 0.0325]     

                                                                                                        \
                                                         

RoBERTa models

In [45]:
model_path = 'CEBaB/roberta-base.CEBaB.sa.2-class.exclusive.seed_99'
# model_path = 'CEBaB/roberta-base.CEBaB.sa.3-class.exclusive.seed_99'
# model_path = 'CEBaB/roberta-base.CEBaB.sa.5-class.exclusive.seed_99'

dataset_type = '2-way'
# dataset_type = '3-way'
# dataset_type = '5-way'

device = 'cuda:0'
batch_size = 32

# load data from HF
cebab = datasets.load_dataset('CEBaB/CEBaB', use_auth_token='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')
cebab['train'] = cebab['train_exclusive']
train, dev, test = preprocess_hf_dataset(cebab, one_example_per_world=False, verbose=1, dataset_type=dataset_type)

# define models
tf_model = RoBERTaForCEBaB(model_path, device=device, batch_size=batch_size)

# fit models
tf_model.fit(train)

models = [
    tf_model,
]

explanators = [
    CONEXP(),
]

final_ATE,final_CEBaB_metrics, final_CEBaB_per_aspect_direction ,final_estimates_per_aspect_direction = run_pipelines(models, explanators, train, dev, dataset_type = dataset_type)

Using custom data configuration CEBaB--CEBaB-65fc31a444dff443
Reusing dataset parquet (/home/kldooste/.cache/huggingface/datasets/parquet/CEBaB--CEBaB-65fc31a444dff443/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train dataset.
Dropped 391 examples with a neutral label.


/project/OpenTable/eval_pipeline/data_utils.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['review_majority'] = dataset['review_majority'].apply(lambda score: encoding[score])
/project/OpenTable/eval_pipeline/data_utils.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[aspect_columns] = train[aspect_columns].fillna('')


Dropped 452 examples with a neutral label.
Dropped 461 examples with a neutral label.


...

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

LSTM

In [47]:
# model_path = 'CEBaB/lstm.CEBaB.sa.2-class.exclusive.seed_99'
# model_path = 'CEBaB/lstm.CEBaB.sa.3-class.exclusive.seed_99'
model_path = 'CEBaB/lstm.CEBaB.sa.5-class.exclusive.seed_99'

# dataset_type = '2-way'
# dataset_type = '3-way'
dataset_type = '5-way'

device = 'cuda:0'
batch_size = 32

# load data from HF
cebab = datasets.load_dataset('CEBaB/CEBaB', use_auth_token='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')
cebab['train'] = cebab['train_exclusive']
train, dev, test = preprocess_hf_dataset(cebab, one_example_per_world=False, verbose=1, dataset_type=dataset_type)

# define models
tf_model = LSTMForCEBaB(model_path, device=device, batch_size=batch_size)

# fit models
tf_model.fit(train)

models = [
    tf_model,
]

explanators = [
    CONEXP(),
]

final_ATE,final_CEBaB_metrics, final_CEBaB_per_aspect_direction ,final_estimates_per_aspect_direction = run_pipelines(models, explanators, train, dev, dataset_type = dataset_type)

Using custom data configuration CEBaB--CEBaB-65fc31a444dff443
Reusing dataset parquet (/home/kldooste/.cache/huggingface/datasets/parquet/CEBaB--CEBaB-65fc31a444dff443/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train dataset.


Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/40.8M [00:00<?, ?B/s]

GPT2

In [49]:
# model_path = 'CEBaB/gpt2.CEBaB.sa.2-class.exclusive.seed_99'
# model_path = 'CEBaB/gpt2.CEBaB.sa.3-class.exclusive.seed_99'
model_path = 'CEBaB/gpt2.CEBaB.sa.5-class.exclusive.seed_99'

# dataset_type = '2-way'
# dataset_type = '3-way'
dataset_type = '5-way'

device = 'cuda:0'
batch_size = 32

# load data from HF
cebab = datasets.load_dataset('CEBaB/CEBaB', use_auth_token='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')
cebab['train'] = cebab['train_exclusive']
train, dev, test = preprocess_hf_dataset(cebab, one_example_per_world=False, verbose=1, dataset_type=dataset_type)

# define models
tf_model = GPT2ForCEBaB(model_path, device=device, batch_size=batch_size)

# fit models
tf_model.fit(train)

models = [
    tf_model,
]

explanators = [
    CONEXP(),
]

final_ATE,final_CEBaB_metrics, final_CEBaB_per_aspect_direction ,final_estimates_per_aspect_direction = run_pipelines(models, explanators, train, dev, dataset_type = dataset_type)

Using custom data configuration CEBaB--CEBaB-65fc31a444dff443
Reusing dataset parquet (/home/kldooste/.cache/huggingface/datasets/parquet/CEBaB--CEBaB-65fc31a444dff443/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

Dropping no majority reviews: 16.6382% of train dataset.


Downloading:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/489M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]